In [ ]:
import pandas as pd
import numpy as np
import re

# GOOGLE MAPS

## Carpeta "metadata-sitios"

Creo variables con el path de cada archivo

In [ ]:
path1 = "metadata-sitios/1.json"
path2= "metadata-sitios/2.json"
path3 = "metadata-sitios/3.json"
path4= "metadata-sitios/4.json"
path5 = "metadata-sitios/5.json"
path6= "metadata-sitios/6.json"
path7 = "metadata-sitios/7.json"
path8= "metadata-sitios/8.json"
path9 = "metadata-sitios/9.json"
path10= "metadata-sitios/10.json"
path11 = "metadata-sitios/11.json"


In [ ]:
# Lista de rutas de archivos JSON
paths = [ path1,path2,path3,
          path4,path5,path6,
          path7,path8,path9,
          path10,path11 ]

In [ ]:

def process_gm_metadata_files(paths):
    result_dataframes = []
# EXTRACCIÓN
    for path in paths:
        # Leer el archivo JSON y convertirlo en DataFrame
        df = pd.read_json(path, lines=True)
        
# TRANSFORMACIÓN
        # Reemplaza la columna "state" y normalizar por el ISO del estado
        df['state'] = df['address'].str.upper()
        df['state'] = df['state'].str.replace('FLORIDA', 'FL')
        df['state'] = df['state'].str.replace('CALIFORNIA', 'CA')
        df['state'] = df['state'].str.replace('TEXAS', 'TX')

        # Revisar los registros que contienen el ISO o el nombre de los estados y reemplazar por el ISO correspondiente
        df['state'] = df.apply(lambda row: 'FL' if row['state'] is not None and ('FL' in row['state'] or 'FLORIDA' in row['state']) else row['state'], axis=1)
        df['state'] = df.apply(lambda row: 'CA' if row['state'] is not None and ('CA' in row['state'] or 'CALIFORNIA' in row['state']) else row['state'], axis=1)
        df['state'] = df.apply(lambda row: 'TX' if row['state'] is not None and ('TX' in row['state'] or 'TEXAS' in row['state']) else row['state'], axis=1)
        
        #Filtrar por los estados determinados
        df = df[df['state'].isin(['CA', 'FL', 'TX'])]
        
        # Lista de palabras clave a buscar
        palabras_clave = ['restaurant', 'cafe', 'cafeteria', 'food', 'fast food', 'fastfood', 'mexican food', 'hamburger']

        # Función para buscar coincidencias de palabras clave en la lista de strings
        def buscar_coincidencias(lista):
            if lista is None:
                return False
            else:
                texto = ' '.join(lista).lower()
                for palabra in palabras_clave:
                    if re.search(r'\b' + re.escape(palabra.lower()) + r'\b', texto):
                        return True
                return False

        # Verificar si alguna palabra clave está presente en las listas de strings de la columna 'category'
        df['contains_keyword'] = df['category'].apply(buscar_coincidencias)      
        # Crear columna y verificar si alguna palabra clave está presente en las listas de strings de la columna 'category'
        #df['contains_keyword'] = df['category'].apply(buscar_coincidencias_rubro_restaurantes)
        
        # Filtrar el DataFrame para obtener solo las filas que contienen palabras clave (los que sean TRUE)
        df = df[df['contains_keyword']]

        # Eliminar la columna 'contains_keyword' si ya no es necesaria
        df.drop('contains_keyword', axis=1, inplace=True)

        # Borrar duplicados de gmap_id:
        df.drop_duplicates(subset="gmap_id", inplace=True)
        
        # Eliminar columnas innecesarias:
        columns = ["description", "hours", "price", "url", "relative_results"]
        df.drop(columns, axis=1, inplace=True)
        
        # Agregar el DataFrame final a la lista de resultados
        result_dataframes.append(df)
        
   
        
    # Une los DataFrames verticalmente
    merged_dataframe = pd.concat(result_dataframes, axis=0)

    # Reiniciar los índices 
    merged_dataframe.reset_index(drop=True, inplace=True)

# CARGA    
    merged_dataframe.to_csv('POST_ETL_DATASETS/gm_metadata.csv', index=False, encoding="utf-8")
    
    return merged_dataframe



In [78]:
gm_metadata= process_gm_metadata_files(paths)

In [176]:
gm_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104968 entries, 0 to 104967
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            104968 non-null  object 
 1   address         104968 non-null  object 
 2   gmap_id         104968 non-null  object 
 3   latitude        104968 non-null  float64
 4   longitude       104968 non-null  float64
 5   category        104968 non-null  object 
 6   avg_rating      104968 non-null  float64
 7   num_of_reviews  104968 non-null  int64  
 8   MISC            103604 non-null  object 
 9   state           104968 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 8.0+ MB


In [249]:
gm_metadata.MISC[0]

{'Service options': ['Takeout', 'Dine-in', 'Delivery'],
 'Accessibility': ['Wheelchair accessible entrance'],
 'Offerings': ['Comfort food'],
 'Amenities': ['Good for kids'],
 'Atmosphere': ['Casual']}

Exporto el dataframe ya limpio a un csv

In [252]:
gm_metadata.to_csv('POST_ETL_DATASETS/gm_metadata.csv', index=False, encoding="utf-8")

## Carpeta "reviews-estados"

Ya que nuestro objetivo general se centra en los estado con mayor pobalción de latinos, el alcance será solo en los estados California , Florida y Texas.



### California

* Creo variables con el paths de cada archivo

In [227]:
cal1 = "reviews-estados/review-California/1.json"
cal2 = "reviews-estados/review-California/2.json"
cal3 = "reviews-estados/review-California/3.json"
cal4 = "reviews-estados/review-California/4.json"
cal5 = "reviews-estados/review-California/5.json"
cal6 = "reviews-estados/review-California/6.json"
cal7 = "reviews-estados/review-California/7.json"
cal8 = "reviews-estados/review-California/8.json"
cal9 = "reviews-estados/review-California/9.json"
cal10 = "reviews-estados/review-California/10.json"
cal11 = "reviews-estados/review-California/11.json"
cal12 = "reviews-estados/review-California/12.json"
cal13 = "reviews-estados/review-California/13.json"
cal14 = "reviews-estados/review-California/14.json"
cal15 ="reviews-estados/review-California/15.json"
cal16 = "reviews-estados/review-California/16.json"
cal17 = "reviews-estados/review-California/17.json"
cal18 = "reviews-estados/review-California/18.json"


* Obtengo el primer dataset en un dataframe

In [181]:
cal_1 = pd.read_json(cal1, lines= True)


KeyboardInterrupt: 

*  Genero una copia del df para poder explorarlo sin modificar nada

In [203]:
cal_1_copy = cal_1.copy()
cal_1_copy.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089912e+20,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


* Reviso su información

In [204]:
cal_1_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     89135 non-null   object 
 5   pics     5632 non-null    object 
 6   resp     16935 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB


De lo anterior:

* Hay muchos nulos en la columna *text* y *resp*; esto conviene reviarlo nuevamente después de unir los datasets de reviews  pertenecientes a cada estado para confirmar si representará un problema.
* La columna *time* debe ser pasada a fecha.
* Filtrar a partir del año 2018 ya que nos interesan los datos más actuales.
* Filtrar solo por restaurantes.
* Columnas innecesarias: *pics* y *name*.

Cambio el tipo de dato de la columna *time* a date_time:

In [205]:
# Especifico la unidad de tiempo en la están los registros (ms) y lo convierto a fecha:
cal_1_copy['time'] = pd.to_datetime(cal_1_copy['time'], unit='ms').dt.date

# Reviso el cambio:
cal_1_copy['time'].head()


0    2021-01-06
1    2021-02-09
2    2020-03-08
3    2019-03-07
4    2017-05-16
Name: time, dtype: object

In [206]:
# Verifico el tipo de dato:
type(cal_1_copy.time[0])

datetime.date

Reviso la fecha más antigua y más reciente

In [207]:
print(cal_1_copy['time'].min())
print(cal_1_copy['time'].max())

2004-04-07
2021-09-09


Reviso cúantos registros hay a partir del año 2018:

In [208]:
cal_1_copy[cal_1_copy['time'].apply(lambda x: x.year)>=2018].shape


(118152, 8)

Filtro a partir del año 2018

In [209]:
cal_1_copy=cal_1_copy[cal_1_copy['time'].apply(lambda x: x.year)>=2018]

Filtro solo los restaurantes

In [210]:
# Genero una lista de los gmap_id de los restaurantes del df gmap_metadata:
lista_id_rest= list(gm_metadata['gmap_id'])


In [211]:
# Filtro los establecimientos de California a partir de la lista anterior:
cal_1_restaurantes = cal_1_copy[cal_1_copy['gmap_id'].isin(lista_id_rest)]



# Reviso el dataframe:
cal_1_restaurantes.head()


,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089912e+20,Song Ro,2021-01-06,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,2021-02-09,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,2020-03-08,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,2019-03-07,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
5,1.001857e+20,Ana Salazar,2019-01-18,5,This food is delicious 😁,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


Por último, elimino las columnas innecesarias:

In [ ]:
cal_1_restaurantes = cal_1_restaurantes.drop(['name','pics'], axis=1)

# Reviso el dataframe:
cal_1_restaurantes.head()
print(cal_1_restaurantes.info())

In [225]:
cal_1_restaurantes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10840 entries, 0 to 149999
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  10840 non-null  float64
 1   time     10840 non-null  object 
 2   rating   10840 non-null  int64  
 3   text     6415 non-null   object 
 4   resp     932 non-null    object 
 5   gmap_id  10840 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 592.8+ KB


Todas las transformaciones necesarias ya están realizadas. A partir de estas creo una función que permita automatizar el ETL de los reviews de cada estado (California, Florida y Texas).

In [219]:
def process_gm_reviews_files(paths, state, df_metadata):
    result_dataframes = []
# EXTRACCIÓN
    for path in paths:
        # Leer el archivo JSON y convertirlo en DataFrame
        df = pd.read_json(path, lines=True)
        
# TRANSFORMACIÓN        
        # Especifico la unidad de tiempo en la están los registros (ms) y lo convierto a fecha:
        df['time'] = pd.to_datetime(df['time'], unit='ms').dt.date

        # Filtro a partir del año 2018:
        df=df[df['time'].apply(lambda x: x.year)>=2018]
        
        # Genero una lista de los gmap_id de los restaurantes del df gmap_metadata:
        lista_id_restaurantes= list(df_metadata['gmap_id'])
        
        # Filtro los establecimientos de California a partir de la lista anterior:
        df = df[df['gmap_id'].isin(lista_id_rest)]
        
        # Eliminar columnas innecesarias:
        df = df.drop(['name','pics'], axis=1)

        # Agregar columna con estado para poder filtrar por ese campo posteriormente:
        df['state']= state
        
        # Agregar el DataFrame final a la lista de resultados
        result_dataframes.append(df)
       
       
    # Une los DataFrames verticalmente
    merged_dataframe = pd.concat(result_dataframes, axis=0)

    # Reiniciar los índices 
    merged_dataframe.reset_index(drop=True, inplace=True)
#CARGA
    merged_dataframe.to_csv(f'POST_ETL_DATASETS/gm_reviews_{state}.csv', index=False, encoding="utf-8")
    
    return merged_dataframe

Creo una lista con los paths de cada json correspondiente a los reviews de California

In [230]:
paths=[cal1, cal2 , cal3, cal4, cal5, cal6, cal7, cal8, cal9,
       cal10, cal11 , cal12, cal13, cal14, cal15, cal16, cal17, cal18]

Aplico la función para extraer, transformar y carga los datasets de California

In [231]:
reviews_CA= process_gm_reviews_files(paths, "CA", gm_metadata)

In [254]:
# Reviso: 
print(reviews_CA.info())
reviews_CA.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651888 entries, 0 to 651887
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  651888 non-null  float64
 1   time     651888 non-null  object 
 2   rating   651888 non-null  int64  
 3   text     364047 non-null  object 
 4   resp     50941 non-null   object 
 5   gmap_id  651888 non-null  object 
 6   state    651888 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 34.8+ MB
None


,user_id,time,rating,text,resp,gmap_id,state
651883,1.129951e+20,2019-07-07,5,None,None,0x80c2956b34fc26c1:0x2f3b0897def449d6,CA
651884,1.023084e+20,2020-01-15,4,None,None,0x80c2956b34fc26c1:0x2f3b0897def449d6,CA
651885,1.125047e+20,2018-10-10,4,None,None,0x80c2956b34fc26c1:0x2f3b0897def449d6,CA
651886,1.021824e+20,2018-08-28,5,None,None,0x80c2956b34fc26c1:0x2f3b0897def449d6,CA
651887,1.155405e+20,2021-01-16,5,None,None,0x80c2956b34fc26c1:0x2f3b0897def449d6,CA


### Florida

* Creo variables con el path de cada archivo

In [257]:
florida1 ="reviews-estados/review-Florida/1.json"
florida2 ="reviews-estados/review-Florida/2.json"
florida3 ="reviews-estados/review-Florida/3.json"
florida4 ="reviews-estados/review-Florida/4.json"
florida5 ="reviews-estados/review-Florida/5.json"
florida6 ="reviews-estados/review-Florida/6.json"
florida7 ="reviews-estados/review-Florida/7.json"
florida8 ="reviews-estados/review-Florida/8.json"
florida9 ="reviews-estados/review-Florida/9.json"
florida10 ="reviews-estados/review-Florida/10.json"
florida11 ="reviews-estados/review-Florida/11.json"
florida12 ="reviews-estados/review-Florida/12.json"
florida13 ="reviews-estados/review-Florida/13.json"
florida14 ="reviews-estados/review-Florida/14.json"
florida15 ="reviews-estados/review-Florida/15.json"
florida16 ="reviews-estados/review-Florida/16.json"
florida17 ="reviews-estados/review-Florida/17.json"
florida18 ="reviews-estados/review-Florida/18.json"
florida19 ="reviews-estados/review-Florida/19.json"


Creo una lista con los paths de los reviews de Florida

In [258]:
paths= [florida1, florida2, florida3, florida4, florida5, florida6, florida7, florida8,florida9, florida10,
        florida11, florida12, florida13, florida14, florida15, florida16, florida17, florida18, florida19]

LLamo a la función para extraer, limpiar y cargar los datos

In [259]:
reviews_FL= process_gm_reviews_files(paths, "FL", gm_metadata)

In [ ]:
# Guardo el archivo
reviews_FL.to_csv(f'POST_ETL_DATASETS/gm_reviews_FL.csv', index=False, encoding="utf-8")

In [260]:
# Reviso: 
print(reviews_FL.info())
reviews_FL.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889304 entries, 0 to 889303
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  889304 non-null  float64
 1   time     889304 non-null  object 
 2   rating   889304 non-null  int64  
 3   text     537991 non-null  object 
 4   resp     127797 non-null  object 
 5   gmap_id  889304 non-null  object 
 6   state    889304 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 47.5+ MB
None


,user_id,time,rating,text,resp,gmap_id,state
889299,1.055903e+20,2018-11-13,5,None,None,0x88e77b72f6649745:0x79b4f1ce48d76510,FL
889300,1.005875e+20,2019-05-19,4,None,"{'time': 1558395889016, 'text': 'Thank you, we...",0x88e77b72f6649745:0x79b4f1ce48d76510,FL
889301,1.102315e+20,2019-08-25,3,None,"{'time': 1566865864044, 'text': 'We will striv...",0x88e77b72f6649745:0x79b4f1ce48d76510,FL
889302,1.056372e+20,2019-12-02,3,None,"{'time': 1575306132300, 'text': 'We will striv...",0x88e77b72f6649745:0x79b4f1ce48d76510,FL
889303,1.137923e+20,2019-04-02,4,None,"{'time': 1554383797677, 'text': 'Thank you for...",0x88e77b72f6649745:0x79b4f1ce48d76510,FL


# FALTA
* Eda de la primera función
* Añadir a la función el desanidado
* (tener en cuenta que el promedio de rating de metadata considera todos los años. (sacar nueva columna?))